# Bidirectional LSTM for non-intrusive reduced order model of a SEIR model in an idealised town

In [1]:
import keras.layers
import numpy as np
import eofs
from eofs.standard import Eof
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow.keras as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import optimizers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from keras.models import save_model
from keras.models import load_model
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 8})
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('axes', labelsize=18)

ModuleNotFoundError: No module named 'keras'

Functions required to: scale and unscaled data, and to obtain lag-n data to train

In [ ]:

def scalerThetis(x, xmin, xmax, min, max):
    scale = (max - min)/(xmax - xmin)
    xScaled = scale*x + min - xmin*scale
    return xScaled

def inverseScalerThetis(xscaled, xmin, xmax, min, max):
    scale = (max - min) / (xmax - xmin)
    xInv = (xscaled/scale) - (min/scale) + xmin
    return xInv

def lookBack(X, look_back = 1):
    #look_back = 10
    X_lb = np.empty((X.shape[0] - look_back + 1, look_back, X.shape[1]))
    #X_test = np.empty((look_back, X_test.shape[1], X_train.shape[0] - look_back + 1))
    ini = 0
    fin = look_back
    for i in range(X.shape[0] - look_back + 1):
        X_lb[i,:,:] = (X[ini:fin,:])
        ini = ini + 1
        fin = fin + 1
    return X_lb


In [ ]:
# Load data
directory = '/data/'
filename = 'group-output-time.csv'

import numpy as np
csv = np.genfromtxt(directory + filename)

ntime = 3888
ngroups = 8
ngridx = 10
ngridy = 10
groups_names = ['HOME - S', 'HOME- E', 'HOME - I', 'HOME - R', 'MOBILE - S', 'MOBILE - E', 'MOBILE - I', 'MOBILE - R']

csv1 = np.reshape(csv, (ntime, ngroups, ngridx, ngridy))
csv2 = np.reshape(csv1, (ntime, ngroups*ngridx*ngridy))

In [ ]:
# Mask out areas where
mask = csv2[500, :].copy()
mask = [np.where(mask == 0, 0, 1)]
mask = np.array(mask)
modeldata = np.zeros((csv2.shape[0], np.count_nonzero(mask)))
for i in range(csv2.shape[0]):
    test = csv2[i, :]
    modeldata[i, :] = test[np.where(mask == 1)[1]]

In [ ]:

# Normalising per variable
sS = modeldata[:, 20*0:20*1]
sE = modeldata[:, 20*1:20*2]
sI = modeldata[:, 20*2:20*3]
sR = modeldata[:, 20*3:20*4]

mS = modeldata[:, 20*4:20*5]
mE = modeldata[:, 20*5:20*6]
mI = modeldata[:, 20*6:20*7]
mR = modeldata[:, 20*7:20*8]

sSmean = np.mean(modeldata[:, 20*0:20*1], axis = 0)
sEmean = np.mean(modeldata[:, 20*1:20*2], axis = 0)
sImean = np.mean(modeldata[:, 20*2:20*3], axis = 0)
sRmean = np.mean(modeldata[:, 20*3:20*4], axis = 0)

mSmean = np.mean(modeldata[:, 20*4:20*5], axis = 0)
mEmean = np.mean(modeldata[:, 20*5:20*6], axis = 0)
mImean = np.mean(modeldata[:, 20*6:20*7], axis = 0)
mRmean = np.mean(modeldata[:, 20*7:20*8], axis = 0)

sSsigma = np.std(modeldata[:, 20*0:20*1])
sEsigma = np.std(modeldata[:, 20*1:20*2])
sIsigma = np.std(modeldata[:, 20*2:20*3])
sRsigma = np.std(modeldata[:, 20*3:20*4])

mSsigma = np.std(modeldata[:, 20*4:20*5])
mEsigma = np.std(modeldata[:, 20*5:20*6])
mIsigma = np.std(modeldata[:, 20*6:20*7])
mRsigma = np.std(modeldata[:, 20*7:20*8])

sSn = (sS - sSmean)/sSsigma
sEn = (sE - sEmean)/sEsigma
sIn = (sI - sImean)/sIsigma
sRn = (sR - sRmean)/sRsigma

mSn = (mS - mSmean)/mSsigma
mEn = (mE - mEmean)/mEsigma
mIn = (mI - mImean)/mIsigma
mRn = (mR - mRmean)/mRsigma

#Concatenate normalised data

modeldataNorm = np.hstack([sSn, sEn, sIn, sRn, mSn, mEn, mIn, mRn])

In [ ]:
#modeldataNorm = (modeldata - meandata)/sigmadata
targetVariance = 0.999
solver = Eof(modeldataNorm)
varianceCumulative = np.cumsum(solver.varianceFraction())
pcs = solver.pcs()
eofs = solver.eofs()
trun = 15
pcs_trun = pcs[:, :trun]
eofs_trun = eofs[:trun, :]

x = np.arange(1,16)
y1 = solver.eigenvalues()[:15]
y2 = varianceCumulative[:15]

In [ ]:
fig, ax1 = plt.subplots(figsize=[20,10])

ax2 = ax1.twinx()
ax1.bar(x, y1)
ax1.semilogy()
ax1.bar(x, y1)

ax1.semilogy()
ax2.plot(x, y2, 'g-o', linewidth=5, markersize=10)
plt.xlim(0,16)
plt.xticks(np.arange(1,16))

ax1.set_xlabel('X data')

ax1.set_ylabel('Eigenvalues', color = 'orange')
ax2.set_ylabel('Cumulative variance', color='g')
plt.xlabel('Component')

In [ ]:
#Reconstruct per variable
trun_data = np.matmul(pcs_trun, eofs_trun)
sStrun = trun_data[:, 20*0:20*1]*sSsigma + sSmean
sEtrun = trun_data[:, 20*1:20*2]*sEsigma + sEmean
sItrun = trun_data[:, 20*2:20*3]*sIsigma + sImean
sRtrun = trun_data[:, 20*3:20*4]*sRsigma + sRmean

mStrun = trun_data[:, 20*4:20*5]*mSsigma + mSmean
mEtrun = trun_data[:, 20*5:20*6]*mEsigma + mEmean
mItrun = trun_data[:, 20*6:20*7]*mIsigma + mImean
mRtrun = trun_data[:, 20*7:20*8]*mRsigma + mRmean

truncated_data = np.hstack([sStrun, sEtrun, sItrun, sRtrun, mStrun, mEtrun, mItrun, mRtrun])
trun_dataf = np.matmul(pcs, eofs)
sStrunf = trun_dataf[:, 20*0:20*1]*sSsigma + sSmean
sEtrunf = trun_dataf[:, 20*1:20*2]*sEsigma + sEmean
sItrunf = trun_dataf[:, 20*2:20*3]*sIsigma + sImean
sRtrunf = trun_dataf[:, 20*3:20*4]*sRsigma + sRmean

mStrunf = trun_dataf[:, 20*4:20*5]*mSsigma + mSmean
mEtrunf = trun_dataf[:, 20*5:20*6]*mEsigma + mEmean
mItrunf = trun_dataf[:, 20*6:20*7]*mIsigma + mImean
mRtrunf = trun_dataf[:, 20*7:20*8]*mRsigma + mRmean

recon_data = np.hstack([sStrunf, sEtrunf, sItrunf, sRtrunf, mStrunf, mEtrunf, mItrunf, mRtrunf])

truncated_masked = np.zeros((csv2.shape[0], csv2.shape[1]))
recon_masked = np.zeros((csv2.shape[0], csv2.shape[1]))

for i in range(csv2.shape[0]):
    test = np.zeros(csv2.shape[1])
    test[np.where(mask == 1)[1]] = truncated_data[i, :]
    truncated_masked[i, :] = test
    test = np.zeros(csv2.shape[1])
    test[np.where(mask == 1)[1]] = recon_data[i, :]
    recon_masked[i, :] = test
trun_shaped = np.reshape(truncated_masked,(ntime, ngroups, ngridx, ngridy))
recon_shaped = np.reshape(recon_masked,(ntime, ngroups, ngridx, ngridy))

In [ ]:
#Fixed point over time
fig = plt.figure(2)
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.plot(csv1[:, i, 0, 5])
    plt.plot(trun_shaped[:, i, 0, 4])
    plt.plot(recon_shaped[:, i, 0, 4])
    plt.title(groups_names[i])

In [ ]:
#PCS every m time-steps
initial_time_step = 0
stepBetween = 10
pcs_stag = pcs_trun[initial_time_step:, :]
pcs_stag = pcs_stag[0:pcs_stag.shape[0]:stepBetween, :]
n_stag_time = pcs_stag.shape[0]

#Staggered original dataset
csv_stag = csv1[initial_time_step:, :, :, :]
csv_stag = csv_stag[0:csv_stag.shape[0]:stepBetween, :, :, :]

#LSTM model
min_ls  = np.min(pcs_stag, 0)
max_ls = np.max(pcs_stag, 0)
min = 0
max = +1

look_backX = 8
look_backY = 1
ls_scaled = scalerThetis(pcs_stag, min_ls, max_ls, min, max)

lsX = np.squeeze(ls_scaled[:-look_backY, :])
lsy = np.squeeze(ls_scaled[look_backY:, :])
X_train, X_test, y_train, y_test = train_test_split(lsX, lsy, test_size=0.1, shuffle=False, random_state=42)

y_train = y_train[look_backX - 1:]
y_test = y_test[look_backX - 1:]

y_train = lookBack(y_train, look_backY)
y_test = lookBack(y_test, look_backY)

X_train = lookBack(X_train, look_backX)
X_test = lookBack(X_test, look_backX)
X_all = lookBack(lsX, look_backX)

In [ ]:
#Create and fit the LSTM network
np.random.seed(42)

#start_time = time()
input_lstm = tf.Input((X_train.shape[1], X_train.shape[2]))
lstm_1 = tf.layers.Bidirectional(tf.layers.LSTM(64, return_sequences=False))(input_lstm)
dropout_1 = tf.layers.Dropout(0.5)(lstm_1)
bn_1 = tf.layers.BatchNormalization()(lstm_1)
rv_1 = tf.layers.RepeatVector(look_backY)(bn_1)
dense_1 = tf.layers.TimeDistributed(tf.layers.Dense(X_train.shape[2], activation='sigmoid'))(rv_1)

lstm_model = tf.Model(input_lstm, dense_1)
lstm_model.summary()

lstm_model.compile(loss='mean_squared_error', metrics=['mae'], optimizer='nadam')
history = lstm_model.fit(X_train, y_train, epochs=500, batch_size=8, verbose=2, validation_data = (X_test, y_test), shuffle = True)

fig = plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
if look_backY == 1:
    predLSTM = lstm_model.predict(X_all)
    predLSTM = np.squeeze(predLSTM)

else:
    timesRepeat = int(np.ceil((X_all.shape[0] - look_backX)/look_backY) + 1)
    predLSTM = np.empty((0,trun))
    for i in range(timesRepeat):
        temp = np.squeeze(lstm_model.predict(np.expand_dims(X_all[i*look_backY, :, :], 0)))
        predLSTM = np.append(predLSTM, temp, axis=0)

predLSTM = inverseScalerThetis(predLSTM, min_ls, max_ls, min, max)
fig = plt.figure()
#Comparison between PCS and predPCS
for i in range(pcs_stag.shape[1]):
    plt.subplot(3,5,i+1)
    plt.plot(np.arange(look_backX, n_stag_time), pcs_stag[look_backX:, i])
    plt.plot(np.arange(look_backX, n_stag_time), predLSTM[:, i])

#predLSTM = np.matmul(predLSTM, eofs_trun)*sigmadata + meandata
predLSTM_masked = np.zeros((predLSTM.shape[0], csv2.shape[1]))
trun_dataL = np.matmul(predLSTM, eofs_trun)
sStrunL = trun_dataL[:, 20*0:20*1]*sSsigma + sSmean
sEtrunL = trun_dataL[:, 20*1:20*2]*sEsigma + sEmean
sItrunL = trun_dataL[:, 20*2:20*3]*sIsigma + sImean
sRtrunL = trun_dataL[:, 20*3:20*4]*sRsigma + sRmean

mStrunL = trun_dataL[:, 20*4:20*5]*mSsigma + mSmean
mEtrunL = trun_dataL[:, 20*5:20*6]*mEsigma + mEmean
mItrunL = trun_dataL[:, 20*6:20*7]*mIsigma + mImean
mRtrunL = trun_dataL[:, 20*7:20*8]*mRsigma + mRmean

predLSTM = np.hstack([sStrunL, sEtrunL, sItrunL, sRtrunL, mStrunL, mEtrunL, mItrunL, mRtrunL])
for i in range(predLSTM.shape[0]):
    test = np.zeros(csv2.shape[1])
    test[np.where(mask == 1)[1]] = predLSTM[i, :]
    predLSTM_masked[i, :] = test
predLSTM_shaped = np.reshape(predLSTM_masked,(n_stag_time-look_backX, ngroups, ngridx, ngridy))

#Fixed point over time
fig = plt.figure(figsize = (20,8))
for i in range(8):
    plt.subplot(4,2,i+1)
    plt.plot(np.arange(look_backX, n_stag_time), csv_stag[look_backX:, i, 0, 4])
    plt.plot(np.arange(look_backX, n_stag_time), predLSTM_shaped[:, i, 0, 4])
    plt.axvline(x=n_stag_time*0.9, color='g')
    plt.title(groups_names[i])
    plt.legend(['Ground truth', 'Prediction'])
plt.tight_layout()